In [1]:
%matplotlib 
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt4Agg


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import sys
sys.path.append('/home/chenani/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [3]:
import signale
import numpy as np
import matplotlib.pyplot as pl
import spectrum as sp
import pickle as pkl
import scipy.signal as scsig
import pywt
import colormaps as mycmps
import signale.tools as tools
import os
import itertools
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
from matplotlib.mlab import  PCA as mlabPCA

/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/home/chenani/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:
colors = ["windows blue", "amber", "greyish", "faded green", "dusty purple"]
seabC = sns.xkcd_palette(colors)
colorsss = ['#FF0000', '#0000FF', '#008000', '#00FFFF', '#FF00FF', '#EE82EE',
        '#808000', '#800080', '#FF6347', '#FFFF00', '#9ACD32', '#4B0082',
        '#FFFACD', '#C0C0C0', '#A0522D', '#FA8072', '#FFEFD5', '#E6E6FA',
        '#F1FAC1', '#C5C5C5', '#A152ED', '#FADD72', '#F0EFD0', '#EEE6FF',
        '#01FAC1', '#F5F5F5', '#A152FF', '#FAFD72', '#F0EFDF', '#EEEFFF',
        '#F1FA99', '#C9C9C9', '#A152DD', '#FA5572', '#FFFFD0', '#EDD6FF']
def zScore(array):
    return(array - np.average(array))/ np.std(array)
clPalette = sns.color_palette('deep')

In [5]:
animalPath = '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/'
dataFolder = '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3906Control/dataFiles/sleepDB/'

In [8]:
lfpPaths = []
for lfp in tools.locate('*-*.lfp',animalPath):
    lfpPaths.append(os.path.join(lfp[0],lfp[1]))
lfpPaths = sorted(lfpPaths)
lfpPaths = [ path for path in lfpPaths if (path.find('l')>0 and path.find('CS')>0) ]
lfpPaths

['/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_05-31-16_Day1/01-sleep1/CSC1-pre.lfp',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_05-31-16_Day1/02-begin1/CSC1-track.lfp',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_05-31-16_Day1/03-sleep2/CSC1-post.lfp',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_06-01-16_Day2/01-sleep1/CSC6-pre.lfp',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_06-01-16_Day2/02-begin1/CSC6-track.lfp',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_06-01-16_Day2/03-sleep2/CSC6-post.lfp',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_06-02-16_Day3/01-sleep1/CSC2-pre.lfp',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_06-02-16_Day3/02-begin1/CSC2-track.lfp',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_06-02

In [9]:
sleepDBPaths = []
for db in tools.locate('Sleep*.db',dataFolder):
    sleepDBPaths.append(os.path.join(db[0],db[1]))
sleepDBPaths = sorted(sleepDBPaths)
sleepDBPaths

['/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3906Control/dataFiles/sleepDB/SleepEpoch-Rat3906-Day1.db',
 '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3906Control/dataFiles/sleepDB/SleepEpoch-Rat3906-Day2.db',
 '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3906Control/dataFiles/sleepDB/SleepEpoch-Rat3906-Day3.db',
 '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3906Control/dataFiles/sleepDB/SleepEpoch-Rat3906-Day4.db']

## Sleep epochs
Loading pre-identified sleep epochs and inserting them into neuralynxCSC objects and saving as .lfp files

In [142]:
rippDB = []
rippPSD = []
whiteRippPSD = []
SWRsignal = []
for ii,path in enumerate(lfpPaths[:]):
    print path
    SleepEpochs = pd.read_pickle(sleepDBPaths[ii/2])
    pickedChannel = pkl.load(open(path,'rb'))
    pickedChannel.tags['file']
    expDate = path.split('_')[2].split('/')[-1]
    animal = path.split('/')[-4]
    session = path.split('/')[-2]
    session = session[:-1]+'0'+session[-1]
    print expDate,session
    Nepochs = SleepEpochs.shape[0]
    pickedChannel.rem_episodes = []
    pickedChannel.sws_episodes = []
    for idx in SleepEpochs.index:
        if SleepEpochs.loc[idx].epoch == 'REM':
            pickedChannel.rem_episodes.append([SleepEpochs.loc[idx].t0+pickedChannel.timeAxis[0],
                                               SleepEpochs.loc[idx].t1+pickedChannel.timeAxis[0]])
        elif SleepEpochs.loc[idx].epoch == 'SWS':
            pickedChannel.sws_episodes.append([SleepEpochs.loc[idx].t0+pickedChannel.timeAxis[0],
                                               SleepEpochs.loc[idx].t1+pickedChannel.timeAxis[0]])
    pickedChannel.rem_episodes = np.array(pickedChannel.rem_episodes)
    pickedChannel.sws_episodes = np.array(pickedChannel.sws_episodes)
    ###################################
    ###########RippDetec
    ###################################
    if (pickedChannel.rem_episodes.size and pickedChannel.sws_episodes.size):
        if not hasattr(pickedChannel,'hilbertAbsolute'):
            print 'Hilbert is busy transforming %s' %pickedChannel.tags['file'].split('/')[-1]
            pickedChannel.hilbertTransform()
        if hasattr(pickedChannel,'ripples') is False:
            print 'Hunting ripples in  %s' %path.split('/')[-1]
            pickedChannel.ripple_recorder(sigma=12, length=12, rippleMix=False,rippleCut=True,SWRmix=True,removeREMripples = False)
    ####################################
    ##########SWR DataFrame
    ####################################
    removeIdx = []
    if pickedChannel.rem_episodes.size and pickedChannel.sws_episodes.size:
        for idx in range(pickedChannel.ripples.shape[0]):
            tP = pickedChannel.ripples[idx,3]
            t1 = tools.findNearest(pickedChannel.timeAxis,tP-50)[0]
            t2 = tools.findNearest(pickedChannel.timeAxis,tP+50)[0]
            if t1!=t2:
                removeIdx.append(idx)
            ripplesMatrix = pickedChannel.ripples[removeIdx,:]
                
        rippDF = pd.DataFrame(ripplesMatrix,columns=['t_start', 't_end','peak value', 't_peak'])
        rippDF['epoch'] = ripplesMatrix.shape[0]*['NA']
        for item in pickedChannel.sws_episodes:
            before = (rippDF.t_peak) < item[1]
            after = (rippDF.t_peak ) > item[0]
            inBetween = np.logical_and(before,after)
            Nep = rippDF['epoch'][inBetween].size
            rippDF['epoch'][inBetween] = Nep * ['sws']
        for item in pickedChannel.rem_episodes:
            before = (rippDF.t_peak) < item[1]
            after = (rippDF.t_peak ) > item[0]
            inBetween = np.logical_and(before,after)
            Nep = rippDF['epoch'][inBetween].size
            rippDF['epoch'][inBetween] = Nep * ['rem']
        rippDF['date'] = rippDF.shape[0]*[expDate]
        rippDF['session'] = rippDF.shape[0]*[session]
        rippDF['animal'] = rippDF.shape[0]*[animal]
        rippDB.append(rippDF.as_matrix().flatten())
    pkl.dump(pickedChannel,open(path,'wb'),pkl.HIGHEST_PROTOCOL)
   
rippDB = np.hstack(rippDB)
rippDB = rippDB.reshape(rippDB.size/8,8)
rippDB = pd.DataFrame(rippDB,
                      columns=['t_i','t_f','peak_value','t_peak','epoch','date','session','animal'])


/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_05-31-16_Day1/01-sleep1/CSC1-pre.lfp
05-31-16

/home/chenani/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/chenani/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


 01-sleep01
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_05-31-16_Day1/03-sleep2/CSC1-post.lfp
05-31-16 03-sleep02
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_06-01-16_Day2/01-sleep1/CSC6-pre.lfp
06-01-16 01-sleep01
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_06-01-16_Day2/03-sleep2/CSC6-post.lfp
06-01-16 03-sleep02
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_06-02-16_Day3/01-sleep1/CSC2-pre.lfp
06-02-16 01-sleep01
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_06-02-16_Day3/03-sleep2/CSC2-post.lfp
06-02-16 03-sleep02
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_06-03-16_Day4/01-sleep1/CSC6-pre.lfp
06-03-16 01-sleep01
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_06-03-16_Day4/03-sleep2/CSC6-post.lfp
06-03-16 03-sleep02


## Frequency and Signal Dataframes

In [209]:
rippPSD = []
whiteRippPSD = []
SWRsignal = []
for path in lfpPaths[:]:
    pickedChannel = pkl.load(open(path,'rb'))
    expDate = path.split('_')[2].split('/')[-1]
    animal = path.split('/')[-4]
    session = path.split('/')[-2]
    session = session[:-1]+'0'+session[-1]
    relevantEpochs = rippDB[np.logical_and(rippDB['date']== expDate,
                                                rippDB['session']==session)]
    
    indexArray = relevantEpochs.index
    print path.split('_')[2].split('/')[-1],path.split('/')[-4],path.split('/')[-2]
    print indexArray
    for idx in indexArray:
        tP = rippDB['t_peak'][idx]
        t1 = tools.findNearest(pickedChannel.timeAxis,tP-50)[0]
        t2 = tools.findNearest(pickedChannel.timeAxis,tP+50)[0]
        if t1!=t2:
            sig = pickedChannel.signal[t1:t2]
            #print sig
            wsig = pickedChannel.signal_white[t1:t2]
        else:
            sig = np.ones(1)
            wsig=np.ones(1)
        p = sp.Periodogram(sig,sampling=2000,NFFT=256)
        wp = sp.Periodogram(wsig,sampling=2000,NFFT=256)
        p();
        wp();
        rippPSD.append(p.psd[:35])
        whiteRippPSD.append(wp.psd[:35])
        SWRsignal.append(sig)
rippPSD = np.array(rippPSD)
whiteRippPSD = np.array(whiteRippPSD)
SWRsignal = np.array(SWRsignal)

05-31-16 Rat3906 01-sleep1
Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            3231, 3232, 3233, 3234, 3235, 3236, 3237, 3238, 3239, 3240],
           dtype='int64', length=3241)
05-31-16 Rat3906 03-sleep2
Int64Index([3241, 3242, 3243, 3244, 3245, 3246, 3247, 3248, 3249, 3250,
            ...
            6151, 6152, 6153, 6154, 6155, 6156, 6157, 6158, 6159, 6160],
           dtype='int64', length=2920)
06-01-16 Rat3906 01-sleep1
Int64Index([6161, 6162, 6163, 6164, 6165, 6166, 6167, 6168, 6169, 6170,
            ...
            8555, 8556, 8557, 8558, 8559, 8560, 8561, 8562, 8563, 8564],
           dtype='int64', length=2404)
06-01-16 Rat3906 03-sleep2
Int64Index([ 8565,  8566,  8567,  8568,  8569,  8570,  8571,  8572,  8573,
             8574,
            ...
            11417, 11418, 11419, 11420, 11421, 11422, 11423, 11424, 11425,
            11426],
           dtype='int64', length=2862)
06-02-16 Rat3906 01-sleep1
Int64Index([11427, 11

In [210]:
pkl.dump(rippPSD,open(dataFolder+'rippPSD.npArr','wb'),pkl.HIGHEST_PROTOCOL)
pkl.dump(whiteRippPSD,open(dataFolder+'whiteRippPSD.npArr','wb'),pkl.HIGHEST_PROTOCOL)
pkl.dump(SWRsignal,open(dataFolder+'rippSignal.npArr','wb'),pkl.HIGHEST_PROTOCOL)

In [211]:
rippDB = pd.read_pickle(dataFolder+'RipplesTimeDB.pd')
rippPSD = pkl.load(open(dataFolder+'rippPSD.npArr','rb'))
whiteRippPSD = pkl.load(open(dataFolder+'whiteRippPSD.npArr','rb'))

In [201]:
pl.scatter(rippDB.c+np.random.rand(rippDB.index.size)/1e2,rippDB.wc+np.random.rand(rippDB.index.size)/1e1)

### Zcoring PSD dataframes

In [212]:
for jjj in range(rippPSD.shape[0]):
    rippPSD[jjj,:] = zScore(np.log10(rippPSD[jjj,:]))
    whiteRippPSD[jjj,:] = zScore(np.log10(whiteRippPSD[jjj,:]))

## PCA of PSD dataframes

Calculating PCA components of SWR power spectrum (both white and original) and adding the first 5 PC components to SWR dataframe!

In [213]:
pca = mlabPCA(rippPSD)
wpca = mlabPCA(whiteRippPSD)
usefulComps = pca.Y[:,range(5)]
wusefulComps = wpca.Y[:,range(5)]

In [214]:
rippDB['PC1'] = usefulComps[:,0]
rippDB['PC2'] = usefulComps[:,1]
rippDB['PC3'] = usefulComps[:,2]
rippDB['PC4'] = usefulComps[:,3]
rippDB['PC5'] = usefulComps[:,4]
rippDB['wPC1'] = wusefulComps[:,0]
rippDB['wPC2'] = wusefulComps[:,1]
rippDB['wPC3'] = wusefulComps[:,2]
rippDB['wPC4'] = wusefulComps[:,3]
rippDB['wPC5'] = wusefulComps[:,4]

In [215]:
rippDB.to_pickle(dataFolder+'RipplesTimeDB.pd')

## Creating Feature files and Clustering

In [216]:
Rkeys =  rippDB.keys()
keys = Rkeys[[8,9,10,11,12]]
wkeys = Rkeys[[13,14,15,16,17]]
features = rippDB[keys].as_matrix()
wfeatures = rippDB[wkeys].as_matrix()
np.savetxt(dataFolder+'pink.fet.1',features,fmt='%f',delimiter='\t',
           header=str(features.shape[1]),comments='')
np.savetxt(dataFolder+'white.fet.1',wfeatures,fmt='%f',delimiter='\t',
           header=str(wfeatures.shape[1]),comments='')

Clustering is done using KlustaKwik program in unix console
## Loading Clusters

In [244]:
c = np.int16(np.loadtxt(dataFolder+'pink.clu.1'))[1:]
wc = np.int16(np.loadtxt(dataFolder+'white.clu.1'))[1:]

In [245]:
cCounts = np.histogram(c,bins=np.arange(0.5,5.5,0.5))[0]
wcCounts = np.histogram(wc,bins=np.arange(0.5,5.5,0.5))[0]
ccc = np.array([cCounts,wcCounts])
c.max(),c.max(),cCounts,wcCounts

(4,
 4,
 array([    0,     1,     0,   637,     0,   109,     0, 21165,     0]),
 array([    0,     0,     0, 21912,     0,     0,     0,     0,     0]))

In [165]:
np.place(wc,wc==1,[100])
np.place(c,c==1,[100])
np.place(wc,wc==3,[1])
np.place(c,c==2,[1])
np.place(c,c==3,[2])
np.place(wc,wc==100,[3])
np.place(c,c==100,[3])

In [189]:
cCounts = np.histogram(c,bins=[0.5,1.5,2.5,3.5])[0]
wcCounts = np.histogram(wc,bins=[0.5,1.5,2.5,3.5])[0]
ccc = np.array([cCounts,wcCounts])
rippDB['wc'] = wc
rippDB['c'] = c

In [266]:
rippDB.to_pickle(dataFolder+'RipplesTimeDB.pd')

In [169]:
f,ax = pl.subplots(1,1)
ax.bar(np.array([1,2,3]), cCounts, 0.35,color= clPalette[0])
ax.bar(np.array([1,2,3]) + 0.35, wcCounts, 0.35,color= clPalette[1])
# add some text for labels, title and axes ticks
ax.set_ylabel('Number of Candidates')
ax.set_title('Clusters')
ax.set_xticks(np.array([1,2,3]) + 0.35)
ax.set_xticklabels(('C1', 'C2','C3'))
ax.set_xlim([00.5,4])

(0.5, 4)

In [264]:
Rcond = np.logical_or(rippDB.wc ==2, rippDB.wc == 8)
Gcond = (rippDB.wc==6)
#Gcond = np.logical_and(rippDB.c ==1, rippDB.wc == 1)
#cond12 = np.logical_and(rippDB.c ==1, rippDB.wc == 2)
#cond21 = np.logical_and(rippDB.c ==2, rippDB.wc == 1)

In [265]:
pl.figure()
sns.tsplot(rippPSD[Rcond.as_matrix(),:],time = p.frequencies()[:35],color='b')
sns.tsplot(rippPSD[Rcond.as_matrix(),:],time = p.frequencies()[:35],color='b',err_style="ci_bars", interpolate=False)
sns.tsplot(rippPSD[Gcond.as_matrix(),:],time = p.frequencies()[:35],color='r')
sns.tsplot(rippPSD[Gcond.as_matrix(),:],time = p.frequencies()[:35],color='r',err_style="ci_bars", interpolate=False)
pl.legend()

/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/axes/_axes.py:519: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [261]:
for ii,item in enumerate([2,6,8]):#enumerate(np.unique(rippDB.wc)):
    indicies = np.where(rippDB.wc==item)[0]
    if indicies.size>200:
        print item,indicies.size,colors[ii]
        sns.tsplot(rippPSD[indicies,:],time = p.frequencies()[:35],color=seabC[ii])

2 758 windows blue
6 16645 amber
8 923 greyish


In [263]:
Gcond

array([ True,  True,  True, ...,  True,  True,  True], dtype=bool)